Developing code to view the defect grids. Eventually, we want this to include some network analysis.

In [1]:
pwd

'/mnt/c/users/jay/desktop/modules/analysis/github/new/packing_defect/notebooks'

In [2]:
import os
import numpy as np
import MDAnalysis as mda
import matplotlib.pyplot as plt


def z_mid(universe, method="box"):
    if method == "box":
        return float(universe.dimensions[2]) * 0.5
    elif method == "data":
        z = universe.atoms.positions[:, 2]
        return 0.5 * (float(z.min()) + float(z.max()))
    else:
        raise ValueError("method must be 'box' or 'data'")

        
def leaflet_mask(universe, leaflet="up", method="box"):
    mid = z_mid(universe, method=method)
    z = universe.atoms.positions[:, 2]
    if leaflet == "up":
        return z > mid
    elif leaflet == "dw":
        return z <= mid
    else:
        raise ValueError("leaflet must be 'up' or 'dw'")

        
        
def leaflet_xy(universe, leaflet="up", method="box"):
    mask = leaflet_mask(universe, leaflet=leaflet, method=method)
    pos = universe.atoms.positions[mask]
    return pos[:, 0], pos[:, 1]



def plot_leaflet(universe, leaflet="up", method="box", s=3, alpha=0.8, ax=None, title=None):
    x, y = leaflet_xy(universe, leaflet=leaflet, method=method)
    if ax is None:
        fig, ax = plt.subplots(figsize=(6,6))
    ax.scatter(x, y, s=s, alpha=alpha)
    Lx, Ly = universe.dimensions[:2]
    if Lx and Ly:
        ax.set_xlim(0, Lx); ax.set_ylim(0, Ly)
    ax.set_aspect("equal", adjustable="box")
    ax.set_xlabel("x (Å)"); ax.set_ylabel("y (Å)")
    if title is None:
        title = f"Defects — leaflet: {leaflet} — center: {method}"
    ax.set_title(title)
    return ax


In [4]:
def frame_path(base_dir, lipid, frame):
    return os.path.join(base_dir, lipid, f"{lipid}_frame_{int(frame)}.gro")



def render_leaflet_frame(base_dir, lipid, frame, leaflet="up", method="box", dpi=150):
    gro = frame_path(base_dir, lipid, frame)
    if not os.path.exists(gro):
        return None

    u = mda.Universe(gro)
    fig, ax = plt.subplots(figsize=(6,6))
    plot_leaflet(u, leaflet=leaflet, method=method, ax=ax,
                 title=f"{lipid} — frame {frame} — {leaflet}")

    outdir = os.path.join(base_dir, f"frames_{lipid}_{leaflet}")
    os.makedirs(outdir, exist_ok=True)
    outpng = os.path.join(outdir, f"{lipid}_{leaflet}_frame_{frame:04d}.png")

    plt.savefig(outpng, dpi=dpi, bbox_inches="tight")
    plt.close(fig)
    return outpng



base_dir = "../results_defect"
for f in range(100):
    render_leaflet_frame(base_dir, "TGglyc", f, leaflet="up", method="box")
